In [1]:
import os
SPARK_HOME="/opt/spark"
os.environ["PYSPARK_PYTHON"] = "python3"
# os.environ["PYSPARK_PYTHON"]="python"
os.environ["PYSPARK_DRIVER_PYTHON"]="jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"]="notebook"
os.environ["SPARK_HOME"] = SPARK_HOME
os.environ["SPARK_CLASSPATH"] = "/opt/spark/jars/sqljdbc4.jar"

 

from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Task_3") \
        .getOrCreate()

In [2]:
#from pyspark.sql import SQLContext
# sqlContext = SQLContext(spark)
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
df_path = "/home/rupeshr/Desktop/TSA_Python/dataset/multivariate_dataset.csv"
df =spark.read.format("com.databricks.spark.csv")\
                    .option("multiline", "true")\
                    .option("header", "true")\
                    .option("inferSchema", "true")\
                    .option("sep", ',')\
                    .option('escape', '\"')\
                    .option("allowSingleQuotes", "true")\
                    .option("ignoreLeadingWhiteSpace", "true")\
                    .option("ignoreTrailingWhiteSpace", "true")\
                    .load(df_path)
df.printSchema()
df.show()

root
 |-- Date: string (nullable = true)
 |-- Money: double (nullable = true)
 |-- Spending: double (nullable = true)
 |-- Gain: integer (nullable = true)
 |-- Loss: integer (nullable = true)

+----------+------+--------+----+----+
|      Date| Money|Spending|Gain|Loss|
+----------+------+--------+----+----+
|1995-01-01|3492.4|  4851.2|2891|1000|
|1995-02-01|3489.9|  4850.8|2893|1101|
|1995-03-01|3491.1|  4885.4|2895|1202|
|1995-04-01|3499.2|  4890.2|2897|1303|
|1995-05-01|3524.2|  4933.1|   0|1404|
|1995-06-01|3548.9|  4977.5|   0|1505|
|1995-07-01|3567.4|  4970.2|   0|1606|
|1995-08-01|3589.0|  5005.3|   0|1707|
|1995-09-01|3602.1|  5020.5|   0|1808|
|1995-10-01|3613.4|  5013.9|   0|1909|
|1995-11-01|3619.9|  5055.6|   0|2010|
|1995-12-01|3629.5|  5097.5|   0|2111|
|1996-01-01|3647.9|  5085.7|   0|2212|
|1996-02-01|3661.9|  5132.8|2917|2313|
|1996-03-01|3687.0|  5173.3|2919|2414|
|1996-04-01|3697.8|  5208.0|2921|   0|
|1996-05-01|3709.7|  5223.8|2923|   0|
|1996-06-01|3722.7|  5229.8

In [4]:
df.describe().show()

+-------+----------+-----------------+------------------+------------------+------------------+
|summary|      Date|            Money|          Spending|              Gain|              Loss|
+-------+----------+-----------------+------------------+------------------+------------------+
|  count|       252|              252|               252|               252|               252|
|   mean|      null|6978.151190476192| 8562.311904761906|3038.1785714285716|13584.436507936507|
| stddev|      null|2554.325114763373|2231.9333524052263| 602.0224313311252| 7511.739127860896|
|    min|1995-01-01|           3489.9|            4850.8|                 0|                 0|
|    max|2015-12-01|          12335.9|           12469.1|              3393|             26351|
+-------+----------+-----------------+------------------+------------------+------------------+



In [ ]:
def auto_fill(username,dsname, column,value=None): # auto fill new
    sqlContext = spark_session(username)
    for i in dsname.columns:
        if i in column:
            s = dsname.select(i).dtypes[0][1]
            if s == "int" or s == "float" or s == "double" or s == "bigint" or s == "long":
                m = "numerical"
                try:
                    #query = "kurtosis(" + i + ") as kurtosis_" + ", skewness(" + i + ") as skewness_" + i + ","
                    #query = query.strip()[:-1]
                    #dsname.registerTempTable("tempTable")
                    #query = "SELECT " + query + " from tempTable"
                    #stats = sqlContext.sql(query).collect()
                    mean = dsname.select(f.avg(i)).collect()[0]
                    mean = mean.asDict()
                    s = "avg(%s)" % i
                    mean = mean[s]
                    median = dsname.approxQuantile(i, [0.5], 0)
                    median = median[0]
                    if stats[0][1] > - 0.5 and stats[0][1] < 0.5:
                        dsname = dsname.na.fill(mean, i)
                    else:
                        dsname = dsname.na.fill(median, i)
                except:
                    dsname = dsname.na.fill(0, i)

            elif s == "string":
                m = "categorical"
                try:
                    dsname = dsname.withColumn(i, f.trim(f.col(i)))
                    if (i == '!' or i == '@' or i == '#' or i == '$' or i == '&' or i == '*' or i == '_' or i == '-' or i == 'NA' or i == '?'):
                        dsname = dsname.withColumn(i, f.when((f.col(i) == "!") | (f.col(i) == "@") | (f.col(i) == '#') | (f.col(i) == '$') | (f.col(i) == '&') | (f.col(i) == '*' | (f.col(i) == '_') | (f.col(i) == '-') | (f.col(i) == 'NA') | (f.col(i) == '?')), None).otherwise(f.col(i)))
                    dsname.registerTempTable("tempTable")
                    modeval = dsname.groupBy(i).count().orderBy(f.desc("count")).collect()
                    temp_iterator = 0
                    temp = modeval[temp_iterator][0]
                    mode = temp
                    if mode == None:
                        mode2 = modeval[1][0]
                        dsname = dsname.na.fill(mode2, i)
                    else:
                        dsname = dsname.na.fill(mode, i)
                except:
                    dsname = dsname.na.fill("0", i)
    return dsname,m

In [10]:
def auto_fill(*,df,column,method,value=None):# auto fill new
    
    if value is not None:
        value=value
    else:
        pass
    
    for i in df.columns:
        if i in column:
            s = df.select(i).dtypes[0]
            print(s)
            if s[1] == "int" or s[1] == "float" or s[1] == "double" or s[1] == "bigint" or s[1] == "long":
                m='numerical'
                try:
                    if method=='mean':
                        mean = df.select(avg(i)).collect()[0]
                        print(mean)
                        mean = mean.asDict()
                        s = "avg(%s)" % i
                        mean = mean[s]
                        dsname = df.na.fill(mean, i)
                    elif method =='median':
                        median = df.approxQuantile(i, [0.5], 0)
                        median = median[0]
                        dsname = df.na.fill(median, i)
                    elif method =='custom':
                        dsname = df.na.fill(value, i)
                except:
                    dsname = df.na.fill(0, i)

            elif s[1] == "string":
                m = "categorical"
                try:
                    if method =='mode':
                        modeval=df.groupBy(i).count().orderBy(desc("count")).collect()
                        temp_iterator = 0
                        temp = modeval[temp_iterator][0]
                        while(temp is None):
                            temp_iterator +=1
                            temp = modeval[temp_iterator][0]
                        mode = temp
                        dsname = df.na.fill(mode, i)
                    elif method =='custom':
                        dsname = df.na.fill(value, i)
                except:
                    dsname = df.na.fill("0", i)

    return dsname,m

In [5]:
df_path = "/home/rupeshr/Desktop/TSA_Python/categorydataset.csv"
df = spark.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load(df_path)
df.show()

+----------+--------+-----+
|      Date|category|count|
+----------+--------+-----+
|1996-01-01|   shade|   23|
|1996-01-02|     sun|   21|
|1996-01-03|     sun| null|
|1996-01-04|    null| null|
|1996-01-05|   shade|   20|
|1996-01-06|     sun| null|
|1996-01-07|    null| null|
|1996-01-08|   shade|   15|
|1996-01-09|     sun|   14|
|1996-01-10|   shade|   17|
|1996-01-11|    null|   12|
|1996-01-12|     sun| null|
|1996-01-13|    null|   11|
|1996-01-14|   shade| null|
|1996-01-15|     sun| null|
+----------+--------+-----+



In [12]:
df1,m=auto_fill(df=df,column='category',method='custom',value='shade')
df1.show()
print(m)

('category', 'string')
+----------+--------+-----+
|      Date|category|count|
+----------+--------+-----+
|1996-01-01|   shade|   23|
|1996-01-02|     sun|   21|
|1996-01-03|     sun| null|
|1996-01-04|   shade| null|
|1996-01-05|   shade|   20|
|1996-01-06|     sun| null|
|1996-01-07|   shade| null|
|1996-01-08|   shade|   15|
|1996-01-09|     sun|   14|
|1996-01-10|   shade|   17|
|1996-01-11|   shade|   12|
|1996-01-12|     sun| null|
|1996-01-13|   shade|   11|
|1996-01-14|   shade| null|
|1996-01-15|     sun| null|
+----------+--------+-----+

categorical


In [35]:
mean = df.select(avg('count')).collect()[0]
print(mean)

Row(avg(count)=16.625)


In [16]:
mean = mean.asDict()
print(mean)                      

{'avg(count)': 16.625}


In [17]:
s = "avg(%s)" % 'count'
print(s)

avg(count)


In [18]:
mean = mean[s]
print(mean)

16.625


In [19]:
dsname = df.na.fill(mean, 'count')
print(dsname)

DataFrame[Date: string, category: string, count: int]


In [20]:
dsname.show()

+----------+--------+-----+
|      Date|category|count|
+----------+--------+-----+
|1996-01-01|   shade|   23|
|1996-01-02|     sun|   21|
|1996-01-03|     sun|   16|
|1996-01-04|    null|   16|
|1996-01-05|   shade|   20|
|1996-01-06|     sun|   16|
|1996-01-07|    null|   16|
|1996-01-08|   shade|   15|
|1996-01-09|     sun|   14|
|1996-01-10|   shade|   17|
|1996-01-11|    null|   12|
|1996-01-12|     sun|   16|
|1996-01-13|    null|   11|
|1996-01-14|   shade|   16|
|1996-01-15|     sun|   16|
+----------+--------+-----+

